|      |      | 
| ---- | ---- | 
| Journal:|Sustainability (ISSN 2071-1050)|
|Manuscript ID:| sustainability-1485080|
|Type:|Article|
|Title:|NEVs Supply Chain Coordination with Financial Constraint and Demand Uncertainty|
|manuscripts:|P.6 - figure 4. Data generation code|
|Code authors:|Yongjian Li$^*$ |
|Last modify:|2021-01-12|
|version:|Python 3.6.5, sympy >= 1.3|

In [1]:
from sympy import *
from sympy.stats import *
import numpy as np
import pandas as pd
# init_printing(use_unicode=True)
import sympy
sympy.__version__

'1.9'

In [2]:
p, w, c, s, B, b, r, alpha, theta, q, a, x, r, mu, sigma, y = symbols(
    "p,w,c,s,B,b,r,alpha,theta,q,a,x,r,mu,sigma,y")

In [3]:
# Initial parameter values
parameter = {p: 10, c: 4, s: 3, w: 6, theta: 0.8, b: 3, B: 100, r: 0.05,
             mu: 100, sigma: 100}

#### (see manuscript P(6) figure 4.)

\begin{align}
&\pi_{r}=q(\theta p-w)-(p \theta-b) \int_{0}^{q} F(x) d x-(w q-B)^+ r\\
&\pi_{s}=q[(1-\theta) p+w-c]-[(1-\theta) p-s+b] \int_{0}^{q} F(x) d x
\end{align}



- Suppose X is normally distributed with $\mu=100$,$\sigma=100$ 
- CDF,exp1: $F(x) = \alpha$

In [4]:
X = Normal(x, mu, sigma).subs(parameter)
exp1 = Eq(cdf(X)(x), alpha)
exp1

Eq(erf(sqrt(2)*(x - 100)/200)/2 + 1/2, alpha)

- inverse exp1: $F^{-1}(\alpha) = x ,\quad \{0\leqslant \alpha \leqslant 1\}$

In [5]:
i_cdf = solve(exp1, x)[0]
i_cdf

100*sqrt(2)*erfinv(2*alpha - 1) + 100

In [117]:
# Convert a SymPy expression into a function that allows for fast numeric evaluation.
f = lambdify(alpha, i_cdf, modules=['numpy', 'sympy'])

x_0 = [i/1000 for i in range(1, 1000)]

dt = {
    "x_0": x_0,
}

df = pd.DataFrame(dt)
df['f(x)'] = df['x_0'].map(f)
df1 = df.copy()
df1['f(x)'] = df1['f(x)'].map(int)
df1 = df1.drop_duplicates(subset=['f(x)']).reset_index(drop=True).copy()
df1

,x_0,f(x)
0,0.001,-209
1,0.002,-187
2,0.003,-174
3,0.004,-165
4,0.005,-157
...,...,...
422,0.995,357
423,0.996,365
424,0.997,374
425,0.998,387


In [7]:
df1.query("`f(x)`==0")

,x_0,f(x)
113,0.157,0


In [8]:
# df1.iloc[::10].to_csv("inverse(CDF)",index=None,sep='\t')

$$q^d_{s}=F^{-1}\left(\frac{\theta p-w(1+r)}{\theta p-b}\right)$$

In [10]:
prob = (p*theta - w*(1+r))/(theta*p - b)
prob

(p*theta - w*(r + 1))/(-b + p*theta)

In [11]:
prob =prob.subs(parameter).evalf(6)
prob

0.340000

In [12]:
eq = Eq(cdf(X)(a), prob)
qd_s = solve(eq, a)[0].subs(parameter)
qd_s = int(qd_s)
qd_s

58

$$q_{t}^{d}=\frac{(\theta p-b) F^{-1}(\alpha)+B(1+r)}{w(1+r)-b}$$

In [119]:
qd_t = ((p*theta-b) *x +B*(1+r))/(w*(1+r)-b)
qd_t

(B*(r + 1) + x*(-b + p*theta))/(-b + w*(r + 1))

In [120]:
qd_t = qd_t.subs(parameter)

f2 = lambdify(x, qd_t, modules=['numpy', 'sympy'])

qd_t_values = f2(df['f(x)'])
qd_t_values = qd_t_values.to_numpy()
qd_t_values.shape

(999,)

In [123]:
dt = {
    "x_0": x_0,
    "f(x)": qd_t_values
}
df_qd_t = pd.DataFrame(dt)
df_qd_t0 = df_qd_t.copy()
df_qd_t['f(x)'] = df_qd_t['f(x)'].map(int)
df_qd_t.head(10)
df_qd_t0

,x_0,f(x)
0,0.001,-284.883682752698
1,0.002,-252.751778650830
2,0.003,-232.997179612877
3,0.004,-218.495425439726
4,0.005,-206.943833871045
...,...,...
994,0.995,573.610500537710
995,0.996,585.162092106391
996,0.997,599.663846279542
997,0.998,619.418445317495


In [27]:
#### df_qd_t = df_qd_t.drop_duplicates(subset=['f(x)']).reset_index(drop=True)
df_qd_t = df_qd_t.iloc[::10].reset_index(drop=True)
df_qd_t

,x_0,f(x)
0,0.001,-284
1,0.011,-163
2,0.021,-124
3,0.031,-98
4,0.041,-78
...,...,...
95,0.951,433
96,0.961,449
97,0.971,469
98,0.981,496


In [28]:
value = solve(Eq(qd_t, qd_s), x)[0]
d1 = {
    "x_0": [cdf(X)(value).evalf()],
    "f(x)": [qd_s]
}
df_qd_t = df_qd_t.append(pd.DataFrame(d1))


value = solve(Eq(qd_t, 0), x)[0]
d1 = {
    "x_0": [cdf(X)(value).evalf()],
    "f(x)": [0]
}

df_qd_t = df_qd_t.append(pd.DataFrame(d1))
df_qd_t = df_qd_t.drop_duplicates(subset=['x_0'])
df_qd_t = df_qd_t.sort_values(by=['x_0']).reset_index(drop=True)
df_qd_t

,x_0,f(x)
0,0.001,-284
1,0.011,-163
2,0.021,-124
3,0.031,-98
4,0.041,-78
...,...,...
97,0.951,433
98,0.961,449
99,0.971,469
100,0.981,496


In [29]:
df_qd_t_e = df_qd_t.query("@qd_s >=`f(x)` >= 0").reset_index(drop=True)
df_qd_t_e

,x_0,f(x)
0,0.113139446443977,0
1,0.121,6
2,0.131,13
3,0.141,21
4,0.151,27
5,0.161,31
6,0.171,37
7,0.181,43
8,0.191,49
9,0.201,56


In [57]:
# df_qd_t.to_csv("qd_t",index=None,sep='\t')
# df_qd_t_e.to_csv("qd_t_e",index=None,sep='\t')

# Retailer's Profit

$\pi_{r}=q(\theta p-w)-(p \theta-b) \int_{0}^{q} F(x) d x-(w q-B)^+ r$

In [124]:
p, w, c, s, B, b, r, alpha, theta, q, a, x, r, mu, sigma, y = symbols(
    "p,w,c,s,B,b,r,alpha,theta,q,a,x,r,mu,sigma,y")

In [125]:
parameter = {p: 10, c: 4, s: 3, w: 6, theta: 0.8, b: 3,
             B: 100, r: 0.05, mu: 100, sigma: 100}

In [126]:
pi_r = q*(p*theta-w) - (p*theta-b)*integrate(cdf(X)(x),(x,0,q)) - Max(0,w*q-B)*r
pi_r

q*(p*theta - w) - r*Max(0, -B + q*w) - (-b + p*theta)*(q*erf(sqrt(2)*q/200 - sqrt(2)/2)/2 + q/2 + 50*sqrt(2)*exp(-1/2)*exp(q/100)*exp(-q**2/20000)/sqrt(pi) - 50*erf(sqrt(2)*q/200 - sqrt(2)/2) - 50*erf(sqrt(2)/2) - 50*sqrt(2)*exp(-1/2)/sqrt(pi))

In [127]:
pi_r = pi_r.subs(parameter)
pi_r

-2.5*q*erf(sqrt(2)*q/200 - sqrt(2)/2) - 0.5*q - 250.0*sqrt(2)*exp(-1/2)*exp(q/100)*exp(-q**2/20000)/sqrt(pi) + 250.0*erf(sqrt(2)*q/200 - sqrt(2)/2) - 0.05*Max(0, 6*q - 100) + 250.0*sqrt(2)*exp(-1/2)/sqrt(pi) + 250.0*erf(sqrt(2)/2)

In [146]:
def fun(exp):
    # 定义np 函数
    f = lambdify(q,exp, modules=['numpy', 'sympy'])
    # 取出F^{-1}(alpha)
    tdf = df_qd_t0.copy()
    tdf['f(x)'] = tdf['f(x)'].map(float)
    tdf['f(x)'] = tdf['f(x)'].map(f)

    # 递增区间的 alpha 值
    a = float(df_qd_t_e['x_0'].min())
    b = float(df_qd_t_e['x_0'].max())
    tdf_e = tdf.query("@b >=`x_0`>= @a").reset_index(drop=True)

    return tdf,tdf_e

In [147]:
pir,pir_in = fun(pi_r)

In [152]:
a = float(df_qd_t_e['x_0'].min())
b = float(df_qd_t_e['x_0'].max())
pir.query("@b >=`x_0`>= @a")

,x_0,f(x)
113,0.114,0.815081840147315
114,0.115,1.74978554990631
115,0.116,2.67126997925871
116,0.117,3.57970992549272
117,0.118,4.47527614433605
...,...,...
199,0.200,33.3737851436830
200,0.201,33.4000556012730
201,0.202,33.4209756769986
202,0.203,33.4365807081204


In [153]:
pir_in

,x_0,f(x)
0,0.114,0.815081840147315
1,0.115,1.74978554990631
2,0.116,2.67126997925871
3,0.117,3.57970992549272
4,0.118,4.47527614433605
...,...,...
86,0.200,33.3737851436830
87,0.201,33.4000556012730
88,0.202,33.4209756769986
89,0.203,33.4365807081204


In [136]:
pir.iloc[::20].to_csv("pir",index=None,sep='\t')
pir_in[::20].to_csv("pir_in",index=None,sep='\t')

In [114]:
pir_in[::30]

,x_0,f(x)
0,0.114,1.20065429820468
30,0.144,21.8064410581165
60,0.174,30.3213691729654
90,0.204,33.4244665350098


# Manufacturer's Profit
$\pi_{s}=q[(1-\theta) p+w-c]-[(1-\theta) p-s+b] \int_{0}^{q} F(x) d x$

In [154]:
p, w, c, s, B, b, r, alpha, theta, q, a, x, r, mu, sigma, y = symbols(
    "p,w,c,s,B,b,r,alpha,theta,q,a,x,r,mu,sigma,y")

In [155]:
pi_s = q*((1-theta)*p+ w -c) - ((1-theta)*p - s + b)*integrate(cdf(X)(x),(x,0,q))
pi_s

q*(-c + p*(1 - theta) + w) - (b + p*(1 - theta) - s)*(q*erf(sqrt(2)*q/200 - sqrt(2)/2)/2 + q/2 + 50*sqrt(2)*exp(-1/2)*exp(q/100)*exp(-q**2/20000)/sqrt(pi) - 50*erf(sqrt(2)*q/200 - sqrt(2)/2) - 50*erf(sqrt(2)/2) - 50*sqrt(2)*exp(-1/2)/sqrt(pi))

In [156]:
parameter = {p: 10, c: 4, s: 3, w: 6, theta: 0.8, b: 3,
             B: 100, r: 0.05, mu: 100, sigma: 100}

In [157]:
pi_s = pi_s.subs(parameter)
pi_s

-1.0*q*erf(sqrt(2)*q/200 - sqrt(2)/2) + 3.0*q - 100.0*sqrt(2)*exp(-1/2)*exp(q/100)*exp(-q**2/20000)/sqrt(pi) + 100.0*erf(sqrt(2)*q/200 - sqrt(2)/2) + 100.0*sqrt(2)*exp(-1/2)/sqrt(pi) + 100.0*erf(sqrt(2)/2)

In [158]:
pi_s.subs({q:58}).evalf(3)

204.

In [159]:
pis,pis_in = fun(pi_s)

In [163]:
pis

,x_0,f(x)
0,0.001,-1122.87440061855
1,0.002,-994.354500388701
2,0.003,-915.348330120842
3,0.004,-857.357779547209
4,0.005,-811.171920467341
...,...,...
994,0.995,1363.88405259483
995,0.996,1386.98725488266
996,0.997,1415.99077578988
997,0.998,1455.49998104413


In [164]:
pis_in

,x_0,f(x)
0,0.114,2.49485893120189
1,0.115,5.37440729590986
2,0.116,8.23322002204219
3,0.117,11.0716212325375
4,0.118,13.8899272297056
...,...,...
86,0.200,196.655196563219
87,0.201,198.459794362783
88,0.202,200.256905479669
89,0.203,202.046598304663


In [169]:
pis.iloc[::20].to_csv("pis",index=None,sep='\t')
pis_in.to_csv("pis_in",index=None,sep='\t')

In [176]:
pis.query("`x_0` == 0.113 ")

,x_0,f(x)
112,0.113,-0.405757281989821


- some help fun() note here.

In [5]:
## note here
## Given a value, calculate the cumulative probability
value = 0
cdf(X)(value).evalf()
## Given a value, calculate the cumulative probability
value = 189.380
P(X < value).evalf(subs=parameter)